In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

### Pretrained model

In [5]:
model = keras.models.load_model('models/saved_model/')

In [6]:
model.trainable = False # Fine-tuning the model.

# # OR
# for layer in model.layers:
#     assert layer.trainable == False
#     layer.trainable = False

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block (ResBlock)        (None, 14, 14, 64)        28640     
                                                                 
 res_block_1 (ResBlock)      (None, 7, 7, 256)         526976    
                                                                 
 res_block_2 (ResBlock)      (None, 3, 3, 512)         1839744   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                             

In [8]:
base_input = model.layers[0].input
base_output = model.layers[-2].output
final_output = layers.Dense(10)(base_output) # Changing to our need, here same

new_model = keras.Model(base_input, final_output)

In [9]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block (ResBlock)        (None, 14, 14, 64)        28640     
                                                                 
 res_block_1 (ResBlock)      (None, 7, 7, 256)         526976    
                                                                 
 res_block_2 (ResBlock)      (None, 3, 3, 512)         1839744   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                             

In [10]:
new_model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [11]:
new_model.fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 2,
    validation_data = (X_test, y_test)
)

Epoch 1/2
1875/1875 [==============================] - 25s 11ms/step - loss: 0.0454 - accuracy: 0.9896 - val_loss: 0.0201 - val_accuracy: 0.9943
Epoch 2/2
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0050 - accuracy: 0.9988 - val_loss: 0.0206 - val_accuracy: 0.9945


### Pretrained keras model

In [28]:
x = tf.random.normal(shape=(5, 299, 299, 3))
y = tf.constant([0, 1, 2, 3, 4])

In [29]:
model = keras.applications.inception_v3.InceptionV3(include_top=True)

In [30]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 149, 149, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_282 (Batch  (None, 149, 149, 32  96         ['conv2d_282[0][0]']             
 Normalization)                 )                                                      

In [31]:
base_input = model.layers[0].input
base_output = model.layers[-2].output
final_output = layers.Dense(5)(base_output)

new_model = keras.Model(base_input, final_output)

In [32]:
new_model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [33]:
new_model.fit(
    x, y,
    batch_size = 32,
    epochs = 10
)

Epoch 1/10
1/1 [==============================] - 12s 12s/step - loss: 1.4970 - accuracy: 0.4000
Epoch 2/10
1/1 [==============================] - 0s 137ms/step - loss: 0.1441 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 138ms/step - loss: 8.0842e-04 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 138ms/step - loss: 1.7997e-04 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 139ms/step - loss: 7.1926e-05 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 137ms/step - loss: 3.9481e-05 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 138ms/step - loss: 2.6917e-05 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 136ms/step - loss: 2.0599e-05 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 135ms/step - loss: 1.7976e-05 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 140ms/step - loss: 1.6713e-05 - accur